# 1. The Qubit

In [6]:
!pip install  -q numpy 2>/dev/null

#  Ensure you have fastcore installed in your Kaggle Notebooks environment
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -U fastcore

## What is a qubit?